# Introdução

In [ ]:
import os
import re
import time
import json
import folium
import random
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
from osgeo import gdal, osr
from bs4 import BeautifulSoup
from tqdm.notebook import trange, tqdm
from geojson2vt.geojson2vt import geojson2vt

In [ ]:
#!pip install geojson2vt

In [ ]:
os.makedirs('maps', exist_ok=True)

<br>

## Merge Dados Tabulares e Espaciais

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp_bombeiro/main/data/tabs/tab_municipios_bombeiros.csv',
)

In [ ]:
# Lê o arquivo csv com o nome dos municípios
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_250k_wgs84.geojson',
)
gdf['id_municipio'] = gdf['id_municipio'].astype(int)

In [ ]:
# Merge
gdf = gdf.merge(
    df,
    on='id_municipio',
    how='left'
)

# Results
gdf.head(3)

## Save *geojson*

In [ ]:
gdf.to_file(
    os.path.join('data', 'shps', 'sp_bombeiros.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

## Folium

In [ ]:
def map_bomb(input_geojson, input_dataframe):
    # I/O
    df = pd.read_csv(input_dataframe)
    gdf = gpd.read_file(input_geojson)
    gdf['id_municipio'] = gdf['id_municipio'].astype(int)

    # Merge
    gdf = gdf.merge(df, on='id_municipio', how='left')

    # Column with category
    col_categories = 'gb_nome'

    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))

    # Create Map
    m = folium.Map([-23, -47], zoom_start=6)
    folium.GeoJson(
        gdf,
        name='Bombeiros',
        smooth_factor=1.0,
        zoom_on_click=True,
        embed=False,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color': color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {'weight': 3},
#         tooltip=folium.features.GeoJsonTooltip(
#             fields=['gb_nome'],
#             aliases=[''],
#             sticky=True,
#             opacity=0.9,
#             direction='right',
#         ),
        popup=folium.Popup(
            'html',
            parse_html=False,
            max_width='100%',
            show=False,
            sticky=False
        )
    ).add_to(m)

    # Plugins
    m.fit_bounds(m.get_bounds())
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
        #force_separate_button=True,
    ).add_to(m)
    return m

In [ ]:
# map_bomb(
#     'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_250k_wgs84.geojson',
#     'https://raw.githubusercontent.com/michelmetran/sp_bombeiro/main/data/tabs/tab_municipios_bombeiros.csv'
# )

map_bomb(
    input_geojson=os.path.join('..', 'sp', 'data', 'shps', 'sp_250k_wgs84.geojson'),
    input_dataframe=os.path.join('data', 'tabs', 'tab_municipios_bombeiros.csv'),
)


# Top FInal Mega ultra!

In [ ]:
# I/O Files
input_geojson=os.path.join('..', 'sp', 'data', 'shps', 'sp_250k_wgs84.geojson')
input_dataframe=os.path.join('data', 'tabs', 'tab_municipios_bombeiros.csv')

In [ ]:
# I/O
df = pd.read_csv(input_dataframe)
gdf = gpd.read_file(input_geojson)
gdf['id_municipio'] = gdf['id_municipio'].astype(int)

# Merge
gdf = gdf.merge(df, on='id_municipio', how='left')

# Add Field
def popup_html(row):
    # Fields
    gb_nome = row['gb_nome']
    comando_nome = row['comando_nome']
    comando_sigla = row['comando_sigla']
    telefone_2 = row['telefone_2']
    
    return """
    <div>
    <h4><b>{}</b></h4>
    <p><b>Sede:</b><br>
    {}{}{}<br>
    {}<br>
    {}<br>
    {}<br>
    <p><b>Contatos:</b><br>
    {}<br>
    {}{}<br></p>
    </div>
    """.format(
        gb_nome,
        '' if row['endereco'] is np.NaN else '{}'.format(row['endereco']),
        '' if row['numero'] is np.NaN else ', {}'.format(row['numero']),
        '' if row['complemento'] is np.NaN or np.isnan(row['complemento']) else ' - {}'.format(row['complemento']),
        '' if row['bairro'] is np.NaN else 'Bairro: {}'.format(row['bairro']),
        '' if row['municipio'] is np.NaN else 'Município: {}'.format(row['municipio']),
        '' if row['cep'] is np.NaN else 'CEP: {}'.format(row['cep']),
        '' if row['email'] is np.NaN else 'E-mail: {}'.format(row['email']),
        '' if row['telefone_1'] is np.NaN else 'Telefone: {}'.format(row['telefone_1']),
        '' if row['telefone_2'] is np.NaN else ' / {}'.format(row['telefone_2']),
    )

gdf['popup'] = gdf.apply(popup_html, axis=1)
gdf.drop([
    'id_municipio', 'endereco', 'numero',
    'complemento', 'bairro', 'municipio', 
    'cep', 'email', 'telefone_1', 'telefone_2',
    'id_gbmar', 'id_gb', 'zona',
    'comando_sigla', 'comando_nome', 
], axis=1, inplace=True)
print(gdf.columns)

# Save geojson
gdf.to_file(
    os.path.join('data', 'shps', 'sp_bombeiros.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

In [ ]:
def convert_geojson2topojson(input_geojson, output_topojson, precision):
    import fiona
    import topojson as tp
    
    # Open and Convert
    with fiona.open(input_geojson) as fio_col:
        topo = tp.Topology(fio_col)

    # Simplify
    topo = topo.toposimplify(precision)

    # Output file
    topo.to_json(output_topojson)

# Convert to TopoJson
convert_geojson2topojson(
    os.path.join('data', 'shps', 'sp_bombeiros.geojson'),
    os.path.join('data', 'shps', 'sp_bombeiros.json'),
    0.001
)

In [ ]:
# Column with category
col_categories = 'gb_nome'

# Set palette
palette_polygon = 'Paired'

# Get list of unique values
categories = set(gdf[col_categories])
categories = list(categories)
categories.sort()

# See the palette chosed
pal = sns.color_palette(palette_polygon, n_colors=len(categories))

# Set dictionary
color_polygon = dict(zip(categories, pal.as_hex()))

In [ ]:
# Cria o mapa
m = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=12,
)

# Adiciona as diferentes empresas com gráficos no popup
lyr = folium.TopoJson(
    open(os.path.join('data', 'shps', 'sp_bombeiros.json')),
    object_path='objects.data',
    name='São Paulo',
    style_function=lambda x: {
        'fillColor': color_polygon[x['properties'][col_categories]],
        'color': color_polygon[x['properties'][col_categories]],
        'weight': 1,
        'fillOpacity': 0.3,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['municipio_nome', 'gb_sigla'],
        aliases=['Munícipio', 'Grupamento'],
        sticky=True,
        opacity=0.9,
        direction='right',
    ),
    #popup='test', # Doesnt Work
)

lyr.add_child(
    folium.Popup(
        #'test',
        gdf['popup'][0],
        #lambda x: x['properties']['popup'],
        #sticky=False,
        #localize=True,
        #parse_html=False,
        max_width=400
    )
)    

# Add Layer to Map
lyr.add_to(m)

# Apresenta o mapa
m.fit_bounds(m.get_bounds())
plugins.Fullscreen(
    position='topleft',
    title='Clique para Maximizar',
    title_cancel='Mininizar',
).add_to(m)
m

## Estudos de Json

In [ ]:
j = json.load(open(os.path.join('data', 'shps', 'sp_bombeiros.json')))
print(j.keys())
print(data.keys())

In [ ]:
j = j['objects']['data']['geometries'][0]
data = data['objects']['data']['geometries'][0]
print(j.keys())
print(data.keys())

In [ ]:
j = j['properties']
data = data['properties']
print(j.keys())
print(data.keys())

## Iterar o Json

In [ ]:
# Cria o mapa
m = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=12,
)
group = folium.FeatureGroup(name='ssss')
j = json.load(open(os.path.join('data', 'shps', 'sp_bombeiros.json')))
for row in j['objects']['data']['geometries']:
    #print(json.dumps({'type': 'Topology', 'objects': {'data': {'geometries': [row]}}}))
    #print(json.dumps(row, indent=4, sort_keys=True))
    #print('---')
    #print(row['properties']['gb_nome'])    
    data = {
        'type': 'Topology',
        'objects': {'data': {'geometries': [row]}},
        'bbox': j['bbox'],
        'transform': j['transform'],
        'arcs': j['arcs'],
    }
    folium.TopoJson(
        data,
        object_path='objects.data'
    ).add_to(group)

group.add_to(m)
LayerControl().add_to(m)
#m.fit_bounds(m.get_bounds())
m

In [ ]:
m

## Topo as 

In [ ]:
data

In [ ]:
m = folium.Map([0, 0], zoom_start=7)
folium.TopoJson(
    data,
    object_path='objects.two_squares'
).add_to(m)
m

In [ ]:
m = folium.Map([0, 0], zoom_start=7)
folium.TopoJson(
    {
      "type":"Topology",
      "transform":{
        "scale": [1,1],
        "translate": [0,0]
      },
      "objects":{ 
        "two_squares":{
          "type": "GeometryCollection",
          "geometries":[
            {"type": "Polygon", "arcs":[[0,1]],"properties": {"name": "Left_Polygon" }},
            {"type": "Polygon", "arcs":[[2,-1]],"properties": {"name": "Right_Polygon" }}
          ]
        }
      },
      "arcs": [
        [[1,2],[0,-2]],
        [[1,0],[-1,0],[0,2],[1,0]],
        [[1,2],[1,0],[0,-2],[-1,0]]
      ]
    },
    object_path='objects.two_squares'
).add_to(m)
m

In [ ]:
m.add_child(group)
m.add_child(folium.LayerControl())
m

In [ ]:
    
#         style_function=lambda x: {
#             'fillColor': color_polygon[x['properties'][col_categories]],
#             'color': color_polygon[x['properties'][col_categories]],
#             'weight': 1,
#             'fillOpacity': 0.3,
#         },
#         tooltip=folium.features.GeoJsonTooltip(
#             fields=['municipio_nome', 'gb_sigla'],
#             aliases=['Munícipio', 'Grupamento'],
#             sticky=True,
#             opacity=0.9,
#             direction='right',
#         ),
        #popup='test', # Doest Work
    

___

# Encerramento

## *Requirements*

In [ ]:
# Input
inp = 'get_infos.ipynb'

# Output
out = os.path.join(inp.split('.')[0])
!jupyter-nbconvert $inp --to python --output $out

print('-'*80)
!pipreqs '.' --force --debug

print('-'*80)
os.remove('{}.py'.format(out))
print('Arquivo {}.py deletado com sucesso!'.format(out))

## *GitHub*

In [ ]:
!nbstripout --install --attributes .gitattributes
!git status
!git add .
!git commit -m 'Arrumações Gerais'
!git push origin main